In [1]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


C:\Users\Vanshika Mishra\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
model= Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [4]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [5]:
train_datagen=ImageDataGenerator(
rescale=1./225,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

In [6]:
test_datagen=ImageDataGenerator(rescale=1./225)

In [7]:
training_set=train_datagen.flow_from_directory(
    'train',
    target_size=(150,150),
    batch_size=16,
    class_mode='binary'
)

Found 7395 images belonging to 2 classes.


In [10]:
test_set=test_datagen.flow_from_directory(
    'test',
    target_size=(150,150),
    batch_size=16,
    class_mode='binary'
)

Found 158 images belonging to 2 classes.


In [11]:
history=model.fit_generator(
training_set,
epochs=20,
validation_data=test_set,
)

C:\Users\Vanshika Mishra\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
10/10 [==============================] - 6s 543ms/step - loss: 0.3754 - accuracy: 0.7848 - val_loss: 0.2712 - val_accuracy: 0.8544
Epoch 2/20
10/10 [==============================] - 4s 350ms/step - loss: 0.2349 - accuracy: 0.8544 - val_loss: 0.1398 - val_accuracy: 0.8544
Epoch 3/20
10/10 [==============================] - 3s 344ms/step - loss: 0.1633 - accuracy: 0.9177 - val_loss: 0.1018 - val_accuracy: 1.0000
Epoch 4/20
10/10 [==============================] - 3s 324ms/step - loss: 0.1219 - accuracy: 0.9747 - val_loss: 0.0674 - val_accuracy: 0.9937
Epoch 5/20
10/10 [==============================] - 3s 309ms/step - loss: 0.0617 - accuracy: 0.9937 - val_loss: 0.0326 - val_accuracy: 0.9937
Epoch 6/20
10/10 [==============================] - 3s 320ms/step - loss: 0.1559 - accuracy: 0.9241 - val_loss: 0.0817 - val_accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 4s 363ms/step - loss: 0.1211 - accuracy: 0.9367 - val_loss: 0.0795 - val_accuracy: 1.0000
Epoch 

In [12]:
model.save('mask_detector',history)

INFO:tensorflow:Assets written to: mask_detector\assets


In [13]:
import numpy as np 
from keras.models import load_model
from keras.preprocessing import image 
import cv2

In [14]:
mymodel=load_model('mask_detector')

In [22]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)

while cap.isOpened():
    s,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors =4)
    
    for(x,y,w,h) in face:
        face_img=img[y:y+h,x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_img=image.load_img('temp.jpg',target_size=(150,150,3))
        test_img= image.img_to_array(test_img)
        test_img=np.expand_dims(test_img,axis=0)
        
        pred=mymodel.predict(test_img)[0][0]
        
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,255,0),-1)
            cv2.putText(img,"No Mask",(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,255,0),-1)
            cv2.putText(img,"Mask",(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('Image',img)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
